In [1]:
import os
import tensorflow as tf
import io
import numpy as np
import tensorflow_addons as tfa

D:\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
D:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
D:\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [8]:
#Define variables

batch_size = 32
img_height = 256
img_width = 256
img_channels = 3

In [9]:
#Normalize pixel function

@tf.autograph.experimental.do_not_convert
def _normalize_img(img, label):
    image = tf.cast(img/255. ,tf.float32)
    return image, label

In [10]:
#Update Data Directory Name

data_dir = 'data_folder/lfw_processed' #Change this to lfw & pubfig dataset

In [11]:
#Load Dataset

lfw_pubfig_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  shuffle=True,
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size
)

lfw_pubfig_ds_norm = lfw_pubfig_ds.map(_normalize_img)

Found 9164 files belonging to 1680 classes.


In [12]:
#Set Dataset Split 0.6/0.2/0.2

train_split = 0.6
val_split = 0.2
test_split = 0.2

In [17]:
#Split Dataset

ds_size = len(list(lfw_pubfig_ds_norm))
train_size = int(train_split * ds_size)
val_size = int(val_split * ds_size)
test_size = int(test_split * ds_size)

train_ds = lfw_pubfig_ds_norm.take(train_size)    
val_ds = lfw_pubfig_ds_norm.skip(train_size).take(val_size)
test_ds = lfw_pubfig_ds_norm.skip(train_size + val_size).take(test_size)

In [32]:
#Print Dataset Sizes

print(len(list(train_ds)) * batch_size, train_ds.element_spec) # Default Batches of 32
print(len(list(val_ds)) * batch_size, val_ds.element_spec) # Default Batches of 32
print(len(list(test_ds)) * batch_size, test_ds.element_spec) # Default Batches of 32

5504 (TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))
1824 (TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))
1824 (TensorSpec(shape=(None, 256, 256, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))


In [45]:
#Create the Baseline Model Class

class BaselineModel(tf.keras.Model):
    def __init__(self):
        super(BaselineModel, self).__init__()
        
        self.input_layer = tf.keras.layers.Input((img_height, img_width, img_channels))
        self.conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=7, padding='same', activation='relu')
        self.mpool1 = tf.keras.layers.MaxPooling2D(pool_size=2)
        
        self.flat1 = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(128, activation=None)

        self.output_layer = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
        
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.mpool1(x)
        x = self.flat1(x)
        x = self.fc1(x)
        out = self.output_layer(x)
        
        return out
    
    def model(self):
        return tf.keras.Model(inputs=self.input_layer, outputs=self.call(self.input_layer))
        

In [ ]:
#Create the Advanced Model Class (Insert Model Architecture)

class AdvancedModel(tf.keras.Model):
    def __init__(self):
        super(AdvancedModel, self).__init__()
        
        self.input_layer = tf.keras.layers.Input((img_height, img_width, img_channels))
        self.conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=7, padding='same', activation='relu')
        self.mpool1 = tf.keras.layers.MaxPooling2D(pool_size=2)
        
        self.flat1 = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(128, activation=None)

        self.output_layer = tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
        
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.mpool1(x)
        x = self.flat1(x)
        x = self.fc1(x)
        out = self.output_layer(x)
        
        return out
    
    def model(self):
        return tf.keras.Model(inputs=self.input_layer, outputs=self.call(self.input_layer))

In [46]:
#Get Model

#Uncomment for Baseline Model
model_name = 'baseline'
model = BaselineModel()

#Uncomment for Advanced Model
#model_name = 'advanced'
#model = AdvancedModel()

model.model().summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_4 (Conv2D)           (None, 256, 256, 64)      9472      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 128, 128, 64)     0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 1048576)           0         
                                                                 
 dense_3 (Dense)             (None, 128)               134217856 
                                                                 
 lambda_3 (Lambda)           (None, 128)               0         
                                                           

In [48]:
#Compliel Model with Adam Optimizer and Triplet Loss

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-2),
    loss=tfa.losses.TripletHardLoss())

In [59]:
#Train Model
EPOCHS = 50

history = model.fit(
        x=train_ds,
        epochs=EPOCHS,
        validation_data=val_ds)

Epoch 1/50
  2/172 [..............................] - ETA: 6:17 - loss: 0.3500

KeyboardInterrupt: 

In [53]:
#Save Model
model.save('saved_model/' + model_name + '_model')

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set

INFO:tensorflow:Assets written to: saved_model/baseline_model\assets


INFO:tensorflow:Assets written to: saved_model/baseline_model\assets


In [52]:
#Save training data
import pandas as pd

# convert the history.history dict to a pandas DataFrame:   
hist_df = pd.DataFrame(history.history) 

# or save to csv: 
hist_csv_file = 'saved_model/' + model_name + '_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [55]:
#Constuct a database of known image encodings from the training/verification dataset as a reference to evaluate the test dataset

pubfig_db_dict = {}
for images, labels in train_ds.take(len(list(train_ds))):
    for i in range(labels.shape[0]):
        if int(labels[i]) not in pubfig_db_dict:
            pubfig_db_dict[int(labels[i])] = model.predict(np.expand_dims(images[i], axis=0))
            
for images, labels in val_ds.take(len(list(val_ds))):
    for i in range(labels.shape[0]):
        if int(labels[i]) not in pubfig_db_dict:
            pubfig_db_dict[int(labels[i])] = model.predict(np.expand_dims(images[i], axis=0))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 71ms/step


1/1 [==============================] - 0s 69ms/step


1/1 [==============================] - 0s 67ms/step


...

1/1 [==============================] - 0s 60ms/step


1/1 [==============================] - 0s 61ms/step


1/1 [==============================] - 0s 87ms/step


1/1 [==============================] - 0s 72ms/step


1/1 [==============================] - 0s 63ms/step


In [57]:
#For Each Image in the test set, compute the closest image embedding
#If it is closer than the threshold, classify the image as that class
#If it is further than the threshold, the test image is not in the referance classes

test_num = 0
test_correct = 0
total_distance = 0

threshold = 1.3

test_samples = len(list(test_ds))

for images, labels in test_ds.take(test_samples):
    for i in range(labels.shape[0]):
        #Establish Target and test encoding
        test_label = int(labels[i])
        test_encoding = model.predict(np.expand_dims(images[i], axis=0))
        
        # Initialize "min_dist" to a large value, say 100 (≈1 line)
        min_dist = 100
        min_label = 0
        
        #Loop through all images in dataset and find closests
        for key in pubfig_db_dict.keys():
            db_enc = pubfig_db_dict[key]
            
            #compute L2 distance
            dist = np.linalg.norm(db_enc - test_encoding)
            
            #set new label if smallest distance seen
            if dist < min_dist:
                min_dist = dist
                min_label = key
                
            if key == test_label:
                total_distance += dist
        
        test_num += 1
        if min_dist < threshold:
            if min_label == test_label:
                test_correct += 1
        elif test_label not in pubfig_db_dict:
            test_correct += 1

1/1 [==============================] - 0s 78ms/step


In [58]:
#Print Facial Recognition Accuracy and Average Distance from correct class

print(test_correct/test_num)
print(total_distance/test_num)

0.0625
1.2289865054190159
